# Retrieve and view profiles for entities
**TODO:** Update how we save results in notebook 6 so there is less redundant code, make profile lookup more efficient, provide python function to lookup profiles for a list of entities simultaneously

### Pre-requisite steps to run this notebook
1. Run the select_final_label_set notebook

In [1]:
import os
import pandas as pd
import numpy as np
import json
from utility import run_command

### Parameters
**REQUIRED**  
**work_dir**: path to work dir that was specified in the select_final_label_set notebook.  
**store_dir**: path to folder containing the sqlite3.db file that we will use for our queries. We will reuse an existing file if there is one in this folder. Otherwise we will create a new one.  
**entity_type**: type of entity you want to look up profiles for. This is just used to navigate to the correct folder that has the profiles.

In [2]:
# **REQUIRED**
work_dir = "../../wikidata_films/profiler_work"
store_dir = "../../wikidata_films"
entity_type = "Q11424" # Film

### Process parameters and set up variables / file names

In [4]:
# Ensure paths are absolute
work_dir = os.path.abspath(work_dir)
store_dir = os.path.abspath(store_dir)

label_set_dir = "{}/final_label_sets/{}".format(work_dir, entity_type)
work_profiles_dir = "{}/work".format(label_set_dir)
final_profiles_dir = "{}/final".format(label_set_dir)
    
# Environment variables
os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(store_dir)
os.environ['kgtk'] = "kgtk" # Need to do this for kgtk to be recognized as a command when passing it through a subprocess call

### Load information that was computed in select_final_label_set notebook

In [5]:
%%time
# load labels into single dataframe
labels_df = pd.read_csv("{}/labels_and_their_entities.tsv".format(final_profiles_dir), delimiter = '\t').fillna("")

# dictionary of label_id --> set of matching entities
# using groupby is not the most efficient, but more concise
label_to_entities = labels_df.groupby("node1")["node2"].apply(set).to_dict()
print("total number of candidate labels for profiling entities of type {}: {}".format(entity_type, len(label_to_entities)))

# final label set (ordered list)
with open("{}/ordered_final_label_set.json".format(final_profiles_dir), 'r') as f:
    label_set = json.load(f)
print("Final label set has {} labels".format(len(label_set)))
# TODO: also add info about how many entities are covered out of total num entities of this type.

# look up number of entities of the type we are profiling
all_ents_in_type = set(pd.read_csv("{}/entities_in_type.tsv".format(work_profiles_dir), delimiter = '\t').entity)
print("Loaded set of all entities of the type we are profiling ({} total)".format(len(all_ents_in_type)))

# Load dictionary of label distinctivenesses incase we want this for analysis
with open("{}/label_distinctiveness_dict.json".format(final_profiles_dir), "r") as f:
    label_distinctiveness = json.load(f)

<decorator-gen-54>:2: DtypeWarning: Columns (11,14,15) have mixed types.Specify dtype option on import or set low_memory=False.


total number of candidate labels for profiling entities of type Q11424: 273
Final label set has 273 labels
Loaded set of all entities of the type we are profiling (176405 total)
CPU times: user 53.3 s, sys: 14.4 s, total: 1min 7s
Wall time: 1min 7s


## View profiles

In [6]:
# NOTE - we can make max_labels_in_profile a notebook param
def print_profile_for_entity(entity, max_labels_in_profile = 5):
    print("Generating profile for entity: {}...".format(entity))

    # We can improve performance using a loop and stopping when we find however many labels we desire.
    # doing it this way here so we can see how many labels total are applicable to the entity
    matching_label_ids_ordered = [l for l in label_set if entity in label_to_entities[l]]

    print("There are {} labels in the final label set that match this entity.".format(len(matching_label_ids_ordered)))
    print("Choosing the top {}:".format(max_labels_in_profile))

    cols_to_display = ["type", "type_label", "prop", "prop_label", "value", "value_label", "value_lb", "value_ub", "prop2", "prop2_label", "value2", "value2_lb", "value2_ub", "si_units", "wd_units"]
    for label_id in matching_label_ids_ordered[:max_labels_in_profile]:
#         print("\nLabel ID: {}".format(label_id))
        display(labels_df.loc[labels_df["node1"]==label_id,cols_to_display].fillna("").iloc[[0]])
#         print("positives for this label: {}".format(", ".join(label_to_entities[label_id])))
        print("Number of positive entities for this label: {} (out of {} entities of this type)".format(len(label_to_entities[label_id]), len(all_ents_in_type)))

Update entities you want to view profiles for below

Casablanca (Q132689):

In [7]:
print_profile_for_entity("Q132689",5)

Generating profile for entity: Q132689...
There are 129 labels in the final label set that match this entity.
Choosing the top 5:


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
5431379,Q11424,'film'@en,P495,'country of origin'@en,Q3624078,'sovereign state'@en,,,P571,'inception'@en,,1675.5,,,


Number of positive entities for this label: 156578 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
198145,Q11424,'film'@en,P462,'color'@en,Q838368,'black-and-white'@en,,,,,,,,,


Number of positive entities for this label: 40690 (out of 176405 entities of this type)


...

Number of positive entities for this label: 54749 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
499695,Q11424,'film'@en,P495,'country of origin'@en,Q1489259,'superpower'@en,,,P1198,'unemployment rate'@en,,,,,Q11229


Number of positive entities for this label: 54749 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
8048996,Q11424,'film'@en,P495,'country of origin'@en,Q5255892,'democratic republic'@en,,,P7295,'Gregorian calendar start date'@en,,1667.0,,,


Number of positive entities for this label: 54749 (out of 176405 entities of this type)


Spirited away (Q155653):  
This one can't be profiled because we only used P31 to define entities' types, and spirited away is an anime film which is a subclass of film. Need to use P279* to profile this one.

In [22]:
"Q155653" in all_ents_in_type

False

Pan's Labrynth (Q216006):

In [24]:
print_profile_for_entity("Q216006",5)

Generating profile for entity: Q216006...
There are 53 labels in the final label set that match this entity.
Choosing the top 5:


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
5114405,Q11424,'film'@en,P495,'country of origin'@en,Q3624078,'sovereign state'@en,,,P571,'inception'@en,,1675.5,,,


Number of positive entities for this label: 156578 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
6833004,Q11424,'film'@en,P495,'country of origin'@en,Q51576574,'Mediterranean country'@en,,,P7295,'Gregorian calendar start date'@en,,,1694.0,,


Number of positive entities for this label: 20486 (out of 176405 entities of this type)


...

Number of positive entities for this label: 20916 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
6673827,Q11424,'film'@en,P495,'country of origin'@en,Q51576574,'Mediterranean country'@en,,,P2573,'number of out-of-school children'@en,,,817705.5,,


Number of positive entities for this label: 20909 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
6609439,Q11424,'film'@en,P495,'country of origin'@en,Q51576574,'Mediterranean country'@en,,,P2131,'nominal GDP'@en,,6.009765e+11,,,Q4917


Number of positive entities for this label: 21768 (out of 176405 entities of this type)


Pulp Fiction (Q104123):

In [18]:
print_profile_for_entity("Q104123",129)

Generating profile for entity: Q104123...
There are 129 labels in the final label set that match this entity.
Choosing the top 129:


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
5114405,Q11424,'film'@en,P495,'country of origin'@en,Q3624078,'sovereign state'@en,,,P571,'inception'@en,,1675.5,,,


Number of positive entities for this label: 156578 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
12009857,Q11424,'film'@en,P495,'country of origin'@en,Q99541706,'historical unrecognized state'@en,,,P2884,'mains voltage'@en,,,,,Q25250


Number of positive entities for this label: 54749 (out of 176405 entities of this type)


...

Number of positive entities for this label: 148411 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
3760952,Q11424,'film'@en,P495,'country of origin'@en,Q3624078,'sovereign state'@en,,,P2219,'real gross domestic product growth rate'@en,,-4.75,9.05,,Q11229


Number of positive entities for this label: 156594 (out of 176405 entities of this type)


,type,type_label,prop,prop_label,value,value_label,value_lb,value_ub,prop2,prop2_label,value2,value2_lb,value2_ub,si_units,wd_units
8805947,Q11424,'film'@en,P495,'country of origin'@en,Q6256,'country'@en,,,P2219,'real gross domestic product growth rate'@en,,-4.75,9.05,,Q11229


Number of positive entities for this label: 157600 (out of 176405 entities of this type)
